<h1 align="center">Statistics for Machine Learning</h1>
<h2 align="center">The Tokenization Pipeline</h2>

&nbsp;

### Overview

Natural language problems deal with textual data which computers cannot immediately understand. For this reason, words (or parts of words) need to be encoded using numbers. These encodings are referred to as *tokens* and can be generated in a number of different ways. The many steps throughout the tokenization pipeline are cruicial to determining the success of a language model, and so this notebook dives deep into the tokenizer methods used in many popular language models toaday. To demonstrate how the theory behind these models works in practice, two different tokenizers have been written from scratch in vanilla Python. These are for the Byte Pair Encoder and WordPiece methods respectively. The notebook finishes by covering how to import and use pre-trained models from the Hugging Face `tokenizers` library, as well as how to train some pre-made models.

&nbsp;


### Contents

Section 1 - Introduction to Tokenization

Section 2 - Normalization and Pre-Tokenization

Section 3 - Subword Tokenization Methods

Section 4 - Tokenizers in Python Libraries

Section 5 - Conclusion

Section 6 - Glossary

Section 7 - Further Reading

### Dependancies

In [2]:
!pip install transformers
!pip install tokenizers

<h2 align="center">Section 1 - Introduction to Tokenization</h2>

### 1.1 - Overview of Tokenizers

Natural language problems concern textual data, which cannot be immediately understood by a machine. For computers to process language, they must first convert the text into a numerical form. This process is carried out in two stages by a model called the **tokenizer**. The tokenizer first takes the text and divides it into smaller pieces, be that words, parts of words, or individual characters. These smaller pieces of text are called **tokens**. The Stanford NLP Group [1] defines tokens more rigorously as:

> an instance of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing.

Once the tokenizer has divided the text into tokens, each token can be assigned a number. An example of this might be that the word 'cat' is assigned the value '15', and so every 'cat' token in the input text will be represented by the number 15. The process of replacing the textual tokens with a number representation is called **encoding**. Similarly, the process of converting encoded tokens back into text is called **decoding**.

&nbsp;

### 1.2 - Different Tokenization Methods

There are three main different methods for dividing text into tokens:

* Word-based
* Character-based
* Subword-based

The following cells give an overview of each of these methods, along with some pros and cons for each.

&nbsp;


### 1.3 - Overview of Word-Based Tokenization Methods

**Word-based tokenization** is perhaps the most simple of the three methods described above. Here, the tokenizer will divide sentences into words by splitting on each space character (sometimes called 'whitespace-based tokenization), or by a similar set of rules (such as punctuation-based tokenization, treebank tokenization, etc) [2].

&nbsp;

For example, the sentence:

&nbsp;

`This sentence is a great, interesting sentence!`

&nbsp;

could be split on whitespace characters to give:

&nbsp;

`['This', 'sentence', 'is', 'a', 'great,', 'interesting', 'sentence!']`

&nbsp;

or by split on both punctuation and spaces to give:

&nbsp;

`['This', 'sentence', 'is', 'a', 'great', ',', 'interesting', 'sentence', '!']`

&nbsp;

From this simple example, it is clear that the rules used to determine the split are important, since the first split gives the potentially rare token 'sentence!', while the second split gives the two, less-rare tokens 'sentence' and '!'. Care should be taken not to remove punctuation altogether, as they can carry very specific meanings. An example of this is the apostrophe, which can distinguish between the plural and possessive form of words. For example "book's" refers to some property of a book, as in "the book's spine is damaged", and "books" refers to many books, as in "the books are damaged".

Once the tokens have been generated, each can be assigned a number. The next time that a token is generated that the tokenizer has already seen, it can simply assign the number that was assigned earlier. For example, if the token 'great' is assigned the value 1 in the sentence above, and the tokenizer is given another sentence that contains the word 'great', this second instance (and all subsequent instances) of the word 'great' will also be assigned the value of 1 [3].

&nbsp;


### 1.4 - Pros and Cons of Word-Based Tokenization Methods

The tokens produced in the word-based method contain a high degree of information, since each token contains semantic and contextual information. However, one of the largest drawbacks with this method is that very similar words are treated as completely separate tokens. For example, the connection between 'cat' and 'cats' would be non-existent, and so these would be treated as separate words. This becomes a problem in large-scale applications that contain many words, as the possible number of tokens in the model's **vocabulary** (the total set of tokens a model has seen) can grow very large. English has around 170,000 words, and so including various grammatical forms like plurals and past-tense for each word can lead to what is known as the **exploding vocabulary problem**. An example of this is the TransformerXL tokenizer which uses whitespace-based splitting. This led to a vocabulary size of over 250,000 [4].

One way to combat this is by enforcing a hard limit on the number of tokens the model can learn (e.g. 10,000). This would classify any word outside of the 10,000 most frequent tokens as **out-of-vocabulary** (OOV), and would assign the token value of 'UNKNOWN' instead of a number value. This causes performance to suffer in cases where many unknown words are present, but may be a suitable compromise if the data contains mostly common words. [3]

&nbsp;

**Summary of Pros:**

* Simple

* High degree of information stored in each token

* Can limit vocabulary size which works well with datasets containing mostly common words


&nbsp;

**Summary of Cons:**

* Separate tokens are created for similar words (e.g. 'cat' and 'cats')

* Can result in very large vocabulary

* Limiting vocabulary can significantly degrade performance on datasets with many uncommon words

&nbsp;


### 1.5 - Overview of Character-Based Tokenization Methods

**Character-based tokenization** splits text on each character, including letters, numbers, and special characters such as punctuation. This greatly reduces the vocabulary size, to the point where the English language can be represented with a vocabulary of around 256 tokens, instead of the 170,000+ needed with word-based approaches [5]. Even east Asian languages such as Chinese and Japanese can see a significant reduction in their vocabulary size, despite containing thousands of unqiue characters in their writing systems.

In a character-based tokenizer, the following sentence:

&nbsp;

`This sentence is a great, interesting sentence!`

&nbsp;

would be converted to:

&nbsp;

`['T', 'h', 'i', 's', ' ', 's', 'e', 'n', 't', 'e', 'n', 'c', 'e', ' ', 'i', 's', ' ', 'a', ' ', 'g', 'r', 'e', 'a', 't', ',', ' ,'i',
'n', 't', 'e', 'r', 'e', 's', 't', 'i', 'n', 'g', ' ', 's', 'e', 'n', 't', 'e', 'n', 'c', 'e', '!'`]

&nbsp;


### 1.6 - Pros and Cons of Character-Based Tokenization Methods

Character-based approaches result in a much smaller vocabulary size when compared to word-based methods, and also result in much fewer out-of-vocabulary tokens. This even allows misspelled words to be tokenized (albeit differently than the correct form of the word), rather than being removed immediately due to the frequency-based vocabulary limit.

However there are a number of drawbacks with this approach too. Firstly, the information stored in a single token produced with a character-based method is very low. This is because unlike the tokens in the word-based method, no semantic or contextual meaning is captured (especially in languages with alphabet-based writing systems like English). Finally, the size of the tokenized input that can be fed into a language model is limited with this approach, since many numbers are required to the input text.

&nbsp;

**Summary of Pros:**

* Smaller vocabulary size

* Does not remove misspelled words

&nbsp;

**Summary of Cons:**

* Low information stored in each token, little-to-no contextual or semantic meaning (especially with alphabet-based writing systems)

* Size of input to language models is limited since the output of the tokenizer requires many more numbers to tokenize text (when compared to a word-based approach)

&nbsp;


### 1.7 - Overview of Subword-Based Tokenization

**Subword-based tokenization** aims to achieve the benefits of both word-based and character-based methods while minimising their downsides. Subword-based methods take a middle ground by splitting text within words in an attempt to create tokens with semantic meaning, even if they are not full words. For example, the tokens `ing` and `ed` carry grammatical meaning, although they are not words in themselve.

This method results in a vocabulary size that is smaller than those found in word-based methods, but larger than those found in character-based methods. The same is also true for the amount of information stored within each token, which is also in between the tokens generated by the previous two methods. The subword approach uses the following two guidelines:

&nbsp;

* Frequently used words should not be split into subwords, but rather be stored as entire tokens

* Infrequently used words should be split into subwords

&nbsp;

Splitting only the infrequently used words gives a chance for the conjugations, plural forms etc to be decomposed into their constiutent parts, while preserving the relationship between tokens. For example `cat` might be a very common word in the dataset, but `cats` might be less common. For this reason, `cats` would be split into `cat` and `s`, where `cat` is now assigned the same value as every other `cat` token, and `s` is assigned a different value, which can encode the meaning of plurality. Another example would be the word `tokenization`, which can be split into the root word `token` and the suffix `ization`. This method can therefore preserve syntactic and semantic similarity [6]. For these reasons, subword-based tokenizers are very commonly used in NLP models today.

&nbsp;


<h2 align="center">Section 2 - Normalization and Pre-Tokenization</h2>



### 2.1 - Overview of the Tokenization Pipeline

The tokenization process requires some pre-processing and post-processing steps, that in all comprise the **tokenization pipeline**. This describes the entires series of actions that are taken to convert raw text into tokens. The steps of this pipeline are:

&nbsp;

* Normalization

* Pre-tokenization

* Model

* Post-processing

&nbsp;

where the tokenization method (be that subword-based, character-based etc) takes place in the model step. [7] This section will cover each of these steps for a tokenizer that uses a subword-based tokenization approach.

&nbsp;

**IMPORTANT NOTE:** all the steps of the tokenization pipeline are handled for the user automaticallly when using a tokenizer from libraries such as the Hugging Face `tokenizers` and `transformers` libraries. The entire pipeline is performed by a single object called the Tokenizer. The cells in this section dive into the inner workings of the code the most users do not need to handle manually when working with NLP tasks. Later in the notebook, the steps to customise the base tokenizer class in the `tokenizers` library is also presented, so that a tokenizer can be purpose-built for a specific task if required.


&nbsp;


### 2.2 - Normalization Methods

**Normalization** is the process of cleaning text before it is split into tokens. This includes converting each character to lowercase, removing accents from characters (e.g. 'é' becomes 'e'), removing unnecessary whitespace, and so on. For example, the string `ThÍs is  áN ExaMPlé     sÉnteNCE` becomes `this is an example sentence` after normalization. Different normalizers will perform different steps, which can be useful depending on the use-case. For example, in some situations the casing or accents might need to be preserved. Depending on the normalizer chosen, different effects can be achieved at this stage.

The Hugging Face `tokenizers.normalizers` package contains several basic normalizers that are used by different tokenizers are part of larger models. The base normalizer class can be imported directly however to investigate how they work. Below shows the NFC unicode, Lowercase, and BERT normalizers. These show the following effects on the example sentence:

&nbsp;

* **NFC:** Does not convert casing or remove accents
* **Lower:** Converts casing but does not remove accents
* **BERT:** Converts casing and removes accents

&nbsp;

In [3]:
from tokenizers.normalizers import NFC, Lowercase, BertNormalizer

# Text to normalize
example_sentence = 'ThÍs is  áN ExaMPlé     sÉnteNCE'

# Instantiate normalizer objects
NFCNorm = NFC()
LowercaseNorm = Lowercase()
BertNorm = BertNormalizer()

# Normalize the text
print(f'NFC:   {NFCNorm.normalize_str(example_sentence)}')
print(f'Lower: {LowercaseNorm.normalize_str(example_sentence)}')
print(f'BERT:  {BertNorm.normalize_str(example_sentence)}')


NFC:   ThÍs is  áN ExaMPlé     sÉnteNCE
Lower: thís is  án examplé     séntence
BERT:  this is  an example     sentence



&nbsp;

The normalizers above are used in tokenizer models which can be imported from the Hugging Face `transformers` library. The cell below shows that the normalizers can be accessed using dot notation via `Tokenizer.backend_tokenizer.normalizer`. Some comparisons are shown between the tokenizsers to highlight the different normalization methods that are used. Note that in these examples, only the FNet normalizer removes unncessary whitespace.

&nbsp;


In [6]:
from transformers import FNetTokenizerFast, CamembertTokenizerFast, BertTokenizerFast

# Text to normalize
example_sentence = 'ThÍs is  áN ExaMPlé     sÉnteNCE'

# Instatiate tokenizers
FNetTokenizer = FNetTokenizerFast.from_pretrained('google/fnet-base')
CamembertTokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')
BertTokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Normalize the text
print(f'FNet Output:      {FNetTokenizer.backend_tokenizer.normalizer.normalize_str(example_sentence)}')
print(f'CamemBERT Output: {CamembertTokenizer.backend_tokenizer.normalizer.normalize_str(example_sentence)}')
print(f'BERT Output:      {BertTokenizer.backend_tokenizer.normalizer.normalize_str(example_sentence)}')

FNet Output:      ThÍs is áN ExaMPlé sÉnteNCE
CamemBERT Output: ThÍs is  áN ExaMPlé     sÉnteNCE
BERT Output:      this is  an example     sentence


### 2.3 - Pre-Tokenization Methods

The **pre-tokenization** step is the first splitting of the raw text in the tokenization pipeline. The split is performed to give an upper bound to what the final tokens could be at the end of the pipeline. That is, a sentence can be split into words in the pre-tokenization step, then in the model step some of these words may be split further according to the tokenization method (e.g. subword-based). So the pre-tokenized text represents the largest possible tokens that could still remain after tokenization.

Just like with normalization, there are several ways that this step can be performed. For example, a sentence can be split based on every space, every space and some punctuation, or every space and every punctuation.

The cell below shows a comparison between the basic `Whitespacesplit` pre-tokenizer, and slightly more complex `BertPreTokenizer` from the Hugging Face `tokenizers.pre_tokenizers` package. The output of the whitespace pre-tokenizer leaves the punctuation in-tact, and still attached to the neighbouring words. For example `includes:` is treated as a single word in this case. Whereas the BERT pre-tokenizer treats punctuation as individual words [8].

&nbsp;


In [7]:
from tokenizers.pre_tokenizers import WhitespaceSplit, BertPreTokenizer

# Text to pre-tokenize - written in lowercase with no unnecessary whitespace to simulate normalization step
example_sentence = "this sentence's content includes: characters, spaces, and punctuation."

# Define helper function to display pre-tokenized output
def print_pretokenized_str(pre_tokens):
    for pre_token in pre_tokens:
        print(f'"{pre_token[0]}", ', end='')

# Instantiate pre-tokenizers
wss = WhitespaceSplit()
bpt = BertPreTokenizer()

# Pre-tokenize the text
print('Whitespace Pre-Tokenizer:')
print_pretokenized_str(wss.pre_tokenize_str(example_sentence))

print('\n\nBERT Pre-Tokenizer:')
print_pretokenized_str(bpt.pre_tokenize_str(example_sentence))


Whitespace Pre-Tokenizer:
"this", "sentence's", "content", "includes:", "characters,", "spaces,", "and", "punctuation.", 

BERT Pre-Tokenizer:
"this", "sentence", "'", "s", "content", "includes", ":", "characters", ",", "spaces", ",", "and", "punctuation", ".", 


&nbsp;

Just as with the normalization methods, you can call the pre-tokenization methods directly from common tokenizers such as the GPT-2 and ALBERT (A Lite BERT) tokenizers. These take a slightly different approach to the standard BERT pre-tokenizer shown above, in that space characters are not removed when splitting the tokens. Instead, they are replaced with special characters that represent where the space was. This has the advantage in that the space characters can be ignored when processing further, but the original sentence can be retrieved if required. The GPT-2 model uses the `Ġ` character, which features a capital 'G' with a dot above. The ALBERT models uses an underscore character.

&nbsp;

In [9]:
from transformers import AutoTokenizer

# Text to pre-tokenize - written in lowercase with no unnecessary whitespace to simulate normalization step
example_sentence = "this sentence's content includes: characters, spaces, and punctuation."

# Instatiate the pre-tokenizers
GPT2_PreTokenizer = AutoTokenizer.from_pretrained('gpt2').backend_tokenizer.pre_tokenizer
Albert_PreTokenizer = AutoTokenizer.from_pretrained('albert-base-v1').backend_tokenizer.pre_tokenizer

# Pre-tokenize the text
print('GPT-2 Pre-Tokenizer:')
print_pretokenized_str(GPT2_PreTokenizer.pre_tokenize_str(example_sentence))
print('\n\nALBERT Pre-Tokenizer:')
print_pretokenized_str(Albert_PreTokenizer.pre_tokenize_str(example_sentence))

GPT-2 Pre-Tokenizer:
"this", "Ġsentence", "'s", "Ġcontent", "Ġincludes", ":", "Ġcharacters", ",", "Ġspaces", ",", "Ġand", "Ġpunctuation", ".", 

ALBERT Pre-Tokenizer:
"▁this", "▁sentence's", "▁content", "▁includes:", "▁characters,", "▁spaces,", "▁and", "▁punctuation.", 


&nbsp;

The cells above show the output of pre-tokenization is a compact format that nicely fits on the screen and removes some of the additional information generated. Below shows the results of a BERT pre-tokenization step on the same example sentence without any of the modifications to the outputs as was shown above. The object returned is a Python list containing tuples. Each tuple corresponds to a pre-token, where the first element is the pre-token string, and the second element is a tuple containing the index for the start and end of the string in the original input text. Note that the starting index of the string is inclusive, and the ending index is exclusive.

&nbsp;


In [13]:
from tokenizers.pre_tokenizers import WhitespaceSplit, BertPreTokenizer

# Text to pre-tokenize - written in lowercase with no unnecessary whitespace to simulate normalization step
example_sentence = "this sentence's content includes: characters, spaces, and punctuation."

# Instantiate pre-tokenizer
bpt = BertPreTokenizer()

# Pre-tokenize the text
bpt.pre_tokenize_str(example_sentence)


[('this', (0, 4)),
 ('sentence', (5, 13)),
 ("'", (13, 14)),
 ('s', (14, 15)),
 ('content', (16, 23)),
 ('includes', (24, 32)),
 (':', (32, 33)),
 ('characters', (34, 44)),
 (',', (44, 45)),
 ('spaces', (46, 52)),
 (',', (52, 53)),
 ('and', (54, 57)),
 ('punctuation', (58, 69)),
 ('.', (69, 70))]

<h2 align="center">Section 3 - Subword Tokenization Methods</h2>



### 3.1 - Subword Tokenization Methods

The model step of the tokenization pipeline is where the tokenization method comes into play. As described earlier, the options here are: word-based, character-based, and subword-based. Subword-based are generally favoured, since these methods were designed to overcome the limitations of the word-based and character-based approaches.

For transformer models, there are three tokenizer methods that are commonly used to implement subword-based tokenization. These include:

&nbsp;

* Byte Pair Encoding (BPE)

* WordPiece

* Unigram

&nbsp;

Each of these use slightly different techniques to split the less frequent words into smaller tokens, which are laid out in this section. In addition, implementations of the BPE and WordPiece algorithms are also shown to help highlight some of the similarities and differences between the approaches.

&nbsp;


### 3.2 - Byte Pair Encoding (BPE) Tokenization

The **Byte Pair Encoding** algorithm is a commonly-used tokenizer that is found in many transformer models such as Open AI's GPT and GPT-2 models, BART, and many others [9-10]. It was originally designed as a text compression algorithm, but has been found to work very well in tokenization tasks for language models. The BPE algorithm decomposes a string of text into subword units that appear frequently in a reference corpus (the text used to train the tokenization model) [11]. The BPE model is trained as follows:

&nbsp;

**Step 1) Construct the Corpus**

The input text is given to the normalization and pre-tokenization models to create clean words. The words are then given to the BPE model, which determines the frequency of each word, and stores this number alongside the word in a list called the **corpus**.

&nbsp;

**Step 2) Construct the Vocabulary**

The words from the corpus are then broken down individual characters and are added to an empty list called the vocabulary. The algorithm will iteratively add to this vocabulary every time it determines which character pairs can be merged together.

&nbsp;

**Step 3) Find the Frequency of Character Pairs**

The frequency of character pairs is then recorded for each word in the corpus. For example, the words `cats` will have the character pairs `ca`, `at`, and `ts`. All the words are examined in this way, and contribute to a global frequency counter. So any instance of `ca` found in any of the tokens will increase the frequency counter for the `ca` pair.

&nbsp;

**Step 4) Create a Merging Rule**

When the frequency for each character pair is known, the most frequent character pair is added to the vocabulary. The vocabulary now consists of every individual letter in the tokens, plus the most frequent character pair. This also gives a merging rule that the model can use. For example, if the model learns that `ca` is the most frequent character pair, it has learned that all adjacent instances of `c` and `a` in the corpus can be merged to give `ca`. This can now be treated as a single character `ca` for the remainder of the steps.

&nbsp;

**Step 5) Repeat Steps 3 and 4**

Steps 3 and are then repeated, finding more merging rules, and adding more character pairs to the vocabulary. This process continues until the vocabulary size reaches a target size specified at the beginning of the training.

&nbsp;

Now that the BPE algorithm has been trained (i.e. now that all have the merging rules have been found), the model can be used to tokenize any text by first splitting each of the words on every character, and then merging according to the merge rules.

&nbsp;


### 3.3 - Implementation of BPE in Python

Below shows a vanilla Python implementation of the BPE algorithm, following the steps outlined above. The following cells show the model trained on a toy dataset, and tested on some example words.

&nbsp;


In [14]:
class TargetVocabularySizeError(Exception):
    def __init__(self, message):
        super().__init__(message)

class BPE:
    '''An implementation of the Byte Pair Encoding tokenizer.'''

    def calculate_frequency(self, words):
        ''' Calculate the frequency for each word in a list of words.

            Take in. list of words stored as strings and return a list of tuples
            where each tuple contains a string from the words list, and an
            integer representing its frequency count in the list.

            Args:
                words (list):  A list of words (strings) in any order.

            Returns:
                corpus (list[tuple(str, int)]):
                               A list of tuples where the first element is a
                               string of a word in the words list, and the
                               second element is an integer representing the
                               frequency of the word in the list.
        '''
        freq_dict = dict()

        for word in words:
            if word not in freq_dict:
                freq_dict[word] = 1
            else:
                freq_dict[word] += 1

        corpus = [(word, freq_dict[word]) for word in freq_dict.keys()]

        return corpus


    def create_merge_rule(self, corpus):
        ''' Create a merge rule and add it to the self.merge_rules list.

            Args:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters (or
                               subwords in later iterations) of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.

            Returns:
                None
        '''
        pair_frequencies = self.find_pair_frequencies(corpus)
        most_frequent_pair = max(pair_frequencies, key=pair_frequencies.get)
        self.merge_rules.append(most_frequent_pair.split(','))
        self.vocabulary.append(most_frequent_pair)


    def create_vocabulary(self, words):
        ''' Create a list of every unique character in a list of words.

            Args:
                words (list): A list of strings containing the words of the
                              input text.

            Returns:
                vocabulary (list): A list of every unique character in the list
                                   of input words.
        '''
        vocabulary = list(set(''.join(words)))
        return vocabulary

    def find_pair_frequencies(self, corpus):
        ''' Find the frequency of each character pair in the corpus.

            Loop through the corpus and calculate the frequency of each pair
            of adjacent characters across every word. Return a dictionary of
            each character pair as the keys and the corresponding frequency as
            the values.

            Args:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters (or
                               subwords in later iterations) of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.

            Returns:
                pair_freq_dict (dict): A dictionary where the keys are the
                                       character pairs from the input corpus and
                                       the values are an integer representing
                                       the frequency of the pair in the corpus.
        '''
        pair_freq_dict = dict()

        for word, word_freq in corpus:
            for idx in range(len(word)-1):

                char_pair = f'{word[idx]},{word[idx+1]}'

                if char_pair not in pair_freq_dict:
                    pair_freq_dict[char_pair] = word_freq
                else:
                    pair_freq_dict[char_pair] += word_freq

        return pair_freq_dict


    def get_merged_chars(self, char_1, char_2):
        ''' Merge the highest score pair and return to the self.merge method.

            This method is abstracted so that the BPE class can be used as the
            base class for other Tokenizers, and so the merging method can be
            easily overwritten. For example, in the BPE algorithm the characters
            can simply be concatenated and returned. However in the WordPiece
            algorithm, the # symbols must first be stripped.

            Args:
                char_1 (str): The first character in the highest-scoring pair.
                char_2 (str): The second character in the highest-scoring pair.

            Returns:
                merged_chars (str): Merged characters.
        '''
        merged_chars = char_1 + char_2
        return merged_chars


    def initialise_corpus(self, words):
        ''' Split each word into characters and count the word frequency.

            Split each word in the input word list on every character. For each
            word, store the split word in a list as the first element inside a
            tuple. Store the frequency count of the word as an integer as the
            second element of the tuple. Create a tuple for every word in this
            fashion and store the tuples in a list called 'corpus', then return
            then corpus list.

            Args:
                None

            Returns:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.
        '''
        corpus = self.calculate_frequency(words)
        corpus = [([*word], freq) for (word, freq) in corpus]
        return corpus


    def merge(self, corpus):
        ''' Loop through the corpus and perform the latest merge rule.

            Args:
                corpus (list[tuple(list, int)]):
                            A list of tuples where the first element is a
                            list of a word in the words list (where the
                            elements are the individual characters (or
                            subwords in later iterations) of the
                            word), and the second element is an integer
                            representing the frequency of the word in the
                            list.

            Returns:
                new_corpus (list[tuple(list, int)]):
                            A modified version of the input argument where the
                            most recent merge rule has been applied to merge
                            the most frequent adjacent characters.
        '''
        merge_rule = self.merge_rules[-1]
        new_corpus = []

        for word, word_freq in corpus:
            new_word = []
            idx = 0

            while idx < len(word):
                # If a merge pattern has been found
                if (len(word) != 1) and (word[idx] == merge_rule[0]) and\
                (word[idx+1] == merge_rule[1]):

                    new_word.append(self.get_merged_chars(word[idx],word[idx+1]))
                    idx += 2
                # If a merge patten has not been found
                else:
                    new_word.append(word[idx])
                    idx += 1

            new_corpus.append((new_word, word_freq))

        return new_corpus


    def train(self, words, target_vocab_size):
        ''' Train the model.

            Args:
                words (list[str]): A list of words to train the model on.

                target_vocab_size (int): The number of words in the vocabulary
                                         to be used as the stopping condition
                                         when training.

            Returns:
                None.
        '''
        self.words = words
        self.target_vocab_size = target_vocab_size
        self.corpus = self.initialise_corpus(self.words)
        self.corpus_history = [self.corpus]
        self.vocabulary = self.create_vocabulary(self.words)
        self.vocabulary_size = len(self.vocabulary)
        self.merge_rules = []

        # Iteratively add vocabulary until the target vocabulary size is reached
        if len(self.vocabulary) > self.target_vocab_size:
            raise TargetVocabularySizeError(f'Error: Target vocabulary size \
            must be greater than the initial vocabulary size \
            ({len(self.vocabulary)})')

        else:
            while len(self.vocabulary) < self.target_vocab_size:
                try:
                    self.create_merge_rule(self.corpus)
                    self.corpus = self.merge(self.corpus)
                    self.corpus_history.append(self.corpus)

                # If no further merging is possible
                except ValueError:
                    print('Exiting: No further merging is possible')
                    break


    def tokenize(self, text):
        ''' Take in some text and return a list of tokens for that text.

            Args:
                text (str): The text to be tokenized.

            Returns:
                tokens (list): The list of tokens created from the input text.
        '''
        tokens = [*text]

        for merge_rule in self.merge_rules:

            new_tokens = []
            idx = 0

            while idx < len(tokens):
                # If a merge pattern has been found
                if (len(tokens) != 1) and (tokens[idx] == merge_rule[0]) and (tokens[idx+1] == merge_rule[1]):

                    new_tokens.append(self.get_merged_chars(tokens[idx],
                                                            tokens[idx+1]))
                    idx += 2
                # If a merge patten has not been found
                else:
                    new_tokens.append(tokens[idx])
                    idx += 1

            tokens = new_tokens

        return tokens



### 3.4 - Training the BPE Algorithm with a Toy Dataset

The BPE algorithm is trained below on a toy dataset that contains some words about cats. The goal of the tokenizer is to determine the most useful, meaningful subunits of the words in the dataset to be used as tokens. From inspection, it is clear that units such as 'cat', 'eat' and 'ing' would be useful subunits.

Running the tokenizer with a target vocabulary size of 21 (which only requires 5 merges) is enough for the tokenizer to capture all the desired subunits mentioned above. With a larger dataset, the target vocabulary would be much higher, but this shows how powerful the BPE tokenizer can be.

&nbsp;


In [15]:
# Training set
words = ['cat', 'cat', 'cat', 'cat', 'cat',
         'cats', 'cats',
         'eat', 'eat', 'eat', 'eat', 'eat', 'eat', 'eat', 'eat', 'eat', 'eat',
         'eating', 'eating', 'eating',
         'running', 'running',
         'jumping',
         'food', 'food', 'food', 'food', 'food', 'food']

# Instantiate the tokenizer
bpe = BPE()
bpe.train(words, 21)

# Print out the corpus at each stage of the process, as well as the merge rule used
print(f'INITIAL CORPUS:\n{bpe.corpus_history[0]}\n')
for rule, corpus in list(zip(bpe.merge_rules, bpe.corpus_history[1:])):
    print(f'NEW MERGE RULE: Combine "{rule[0]}" and "{rule[1]}"')
    print(corpus, end='\n\n')

INITIAL CORPUS:
[(['c', 'a', 't'], 5), (['c', 'a', 't', 's'], 2), (['e', 'a', 't'], 10), (['e', 'a', 't', 'i', 'n', 'g'], 3), (['r', 'u', 'n', 'n', 'i', 'n', 'g'], 2), (['j', 'u', 'm', 'p', 'i', 'n', 'g'], 1), (['f', 'o', 'o', 'd'], 6)]

NEW MERGE RULE: Combine "a" and "t"
[(['c', 'at'], 5), (['c', 'at', 's'], 2), (['e', 'at'], 10), (['e', 'at', 'i', 'n', 'g'], 3), (['r', 'u', 'n', 'n', 'i', 'n', 'g'], 2), (['j', 'u', 'm', 'p', 'i', 'n', 'g'], 1), (['f', 'o', 'o', 'd'], 6)]

NEW MERGE RULE: Combine "e" and "at"
[(['c', 'at'], 5), (['c', 'at', 's'], 2), (['eat'], 10), (['eat', 'i', 'n', 'g'], 3), (['r', 'u', 'n', 'n', 'i', 'n', 'g'], 2), (['j', 'u', 'm', 'p', 'i', 'n', 'g'], 1), (['f', 'o', 'o', 'd'], 6)]

NEW MERGE RULE: Combine "c" and "at"
[(['cat'], 5), (['cat', 's'], 2), (['eat'], 10), (['eat', 'i', 'n', 'g'], 3), (['r', 'u', 'n', 'n', 'i', 'n', 'g'], 2), (['j', 'u', 'm', 'p', 'i', 'n', 'g'], 1), (['f', 'o', 'o', 'd'], 6)]

NEW MERGE RULE: Combine "i" and "n"
[(['cat'], 5), (['cat'

### 3.5 - Tokenizing using BPE

Now that the BPE algorithm has been trained with this very small dataset, it can be used to tokenize some example words. The cell below shows the tokenizer being used to tokenize some words it has seen before, and some it has not. Notice that the tokenizer has learned the verb ending `ing`, and so can split these characters into a token, even in words it has never seen before. In the case of `eating`, the training data contained the word `eat`, and so the model was able to learn that this is a significant token all by itself. However, the model has never seen the words `run` and `ski`, and so does not successfully tokenize these as root words. This highlights the important of a wide and varied training set when training a tokenizer.

&nbsp;

In [16]:
print(bpe.tokenize('eating'))
print(bpe.tokenize('running'))
print(bpe.tokenize('skiing'))


['eat', 'ing']
['r', 'u', 'n', 'n', 'ing']
['s', 'k', 'i', 'ing']


### 3.6 - The Byte-Level BPE Trick

BPE tokenizers can only recognise characters that have appeared in the training data. For example, the training data above only contained the characters needed to talk about cats, which happened to not require a `z`. Therefore, that version of the tokenizer does not contain the character `z` in its vocabulary and so would convert that character to an unknown token if the model was used to tokenize real data (actually, error handling was not added to instruct the model to create unknown tokens and so it would crash, but for productionised models this is the case).

The BPE tokenizers used in GPT-2 and RoBERTa do not have this issue due to a trick within the code. Instead of analysing the training data based on the Unicode characters, they instead analyse the character's bytes. This is called **Byte-Level BPE** and allows a small base vocabulary to be able to tokenize all characters the model might see.

&nbsp;


### 3.7 - WordPiece Tokenization

**WordPiece** is a tokenization method developed by Google for their seminal BERT model, and is used in its derivative models such as DistilBERT and MobileBERT.

The full details of the WordPiece algorithm have not been fully released to the public, and so the methodology presented in this notebook is based on the interpretation given by Hugging Face [12]. The WordPiece algorithm is similiar to BPE, but uses a different metric to determine the merge rules. Instead of choosing the most frequent character pair, a score is calculated for each pair instead, and the pair with the highest score determines which characters are merged. WordPiece is trained as follows:

&nbsp;

**Step 1) Construct the Corpus**

Again, the input text is given to the normalization and pre-tokenization models to create clean words. The words are then given to the WordPiece model, which determines the frequency of each word, and stores this number alongside the word in a list called the corpus.

&nbsp;

**Step 2) Construct the Vocabulary**

As with BPE, the words from the corpus are then broken down individual characters and are added to an empty list called the vocabulary. However this time, instead of storing simply each individual character, two # symbols are used as markers to determine whether the character was found at the beginning of a word, or from the middle/end of a word. For example, the word `cat` would be split into `['c', 'a', 't']` in BPE, but in WordPiece it would look like `['c', '##a', '##t']`. In this system, `c` at the beginning of a word, and `##c` from the middle or end of a word, would be treated differently. As before, he algorithm will iteratively add to this vocabulary every time it determines which character pairs can be merged together.

&nbsp;

**Step 3) Calculate the Pair Score for Each Adjacent Character Pair**

Unlike the BPE model, this time a score is calculated for each of the character pairs. First, each adjacent character pair in the corpus is identified, e.g. `'c##a'`, `##a##t` and so on, and the frequency is counted. The frequency of each character individually is also determined. With these values known, a pair score can then be calculated according to the following formula:

&nbsp;

$
    \begin{align}
    \frac{\text{freq of pair}}{\text{freq of first element} \space \cdot{} \space{} \text{freq of second element}}
    \end{align}
$

&nbsp;

This metric assigns higher scores to characters that appear frequently together, but less frequently on their own or with other characters. This is the main difference between WordPiece and BPE, since BPE does not take into account the overall frequency of the individual characters on their own.

&nbsp;

**Step 4) Create a Merging Rule**

High scores represent character pairs that are commonly seen together. That is, if `c##a` has a high pair score, then `c` and `a` are frequently seen together in the corpus, and not so frequently seen separately. Just as with BPE, the merging rule is determined by taking the character pair with the highest score, but this time instead of frequency determining the score, its the pair score.

&nbsp;

**Step 5) Repeat Steps 3 and 4**

Steps 3 and are then repeated, finding more merging rules, and adding more character pairs to the vocabulary. This process continues until the vocabulary size reaches a target size specified at the beginning of the training.

&nbsp;

The cell below shows an implementation of WordPiece which inherits from the BPE model written earlier.

&nbsp;

In [17]:
class WordPiece(BPE):

    def add_hashes(self, word):
        ''' Add # symbols to every character in a word except the first.

            Take in a word as a string and add # symbols to every character
            except the first. Return the result as a list where each element is
            a character with # symbols in front, except the first character
            which is just the plain character.

            Args:
                word (str): The word to add # symbols to.

            Returns:
                hashed_word (list): A list of the characters with # symbols
                                    (except the first character whihc is just
                                    the plain character).
        '''
        hashed_word = [word[0]]

        for char in word[1:]:
            hashed_word.append(f'##{char}')

        return hashed_word


    def create_merge_rule(self, corpus):
        ''' Create a merge rule and add it to the self.merge_rules list.

            Args:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters (or
                               subwords in later iterations) of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.

            Returns:
                None
        '''
        pair_frequencies = self.find_pair_frequencies(corpus)
        char_frequencies = self.find_char_frequencies(corpus)
        pair_scores = self.find_pair_scores(pair_frequencies, char_frequencies)

        highest_scoring_pair = max(pair_scores, key=pair_scores.get)
        self.merge_rules.append(highest_scoring_pair.split(','))
        self.vocabulary.append(highest_scoring_pair)


    def create_vocabulary(self, words):
        ''' Create a list of every unique character in a list of words.

            Unlike the BPE algorithm where each character is stored normally,
            here a distinction is made by characters that begin a word
            (unmarked), and characters that are in the middle or end of a word
            (marked with a '##'). For example, the word 'cat' will be split into
            ['c', '##a', '##t'].

            Args:
                words (list): A list of strings containing the words of the
                                input text.

            Returns:
                vocabulary (list): A list of every unique character in the list
                                    of input words, marked accordingly with ##
                                    to denote if the character was featured in
                                    the middle/end of a word, instead of as the
                                    first character of the word.
        '''
        vocabulary = set()
        for word in words:
            vocabulary.add(word[0])
            for char in word[1:]:
                vocabulary.add(f'##{char}')

        # Convert to list so the vocabulary can be appended to later
        vocabulary = list(vocabulary)
        return vocabulary


    def find_char_frequencies(self, corpus):
        ''' Find the frequency of each character in the corpus.

            Loop through the corpus and calculate the frequency of characters.
            Note that 'c' and '##c' are different characters, since the first
            represents a 'c' at the start of a word, and '##c' represents a 'c'
            in the middle/end of a word. Return a dictionary of each character
            pair as the keys and the corresponding frequency as the values.

            Args:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters (or
                               subwords in later iterations) of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.

            Returns:
                pair_freq_dict (dict): A dictionary where the keys are the
                                       characters from the input corpus and the
                                       values are an integer representing
                                       the frequency.
        '''
        char_frequencies = dict()

        for word, word_freq in corpus:
            for char in word:
                if char in char_frequencies:
                    char_frequencies[char] += word_freq
                else:
                    char_frequencies[char] = word_freq

        return char_frequencies


    def find_pair_scores(self, pair_frequencies, char_frequencies):
        ''' Find the pair score for each character pair in the corpus.

            Loops through the pair_frequencies dictionary and calculate the pair
            score for each pair of adjacent characters in the corpus. Store the
            scores in a dictionary and return it.

            Args:
                pair_frequencies (dict):
                               A dictionary where the keys are the adjacent
                               character pairs in the corpus and the values are
                               the frequencies of each pair.

                char_frequencies (dict):
                               A dictionary where the keys are the characters in
                               the corpus and the values are corresponding
                               frequencies.

            Returns:
                pair_scores (dict): A dictionary where the keys are the
                                    adjacent character pairs in the input corpus
                                    and the values are the corresponding pair
                                    score.
        '''
        pair_scores = dict()

        for pair in pair_frequencies.keys():
            char_1 = pair.split(',')[0]
            char_2 = pair.split(',')[1]
            denominator = (char_frequencies[char_1]*char_frequencies[char_2])
            score = (pair_frequencies[pair]) / denominator
            pair_scores[pair] = score

        return pair_scores


    def get_merged_chars(self, char_1, char_2):
        ''' Merge the highest score pair and return to the self.merge method.

            Remove the # symbols as necessary and merge the highest scoring pair
            then return the merged characters to the self.merge method.


            Args:
                char_1 (str): The first character in the highest-scoring pair.
                char_2 (str): The second character in the highest-scoring pair.

            Returns:
                merged_chars (str): Merged characters.
        '''
        if char_2.startswith('##'):
            merged_chars = char_1 + char_2[2:]
        else:
            merged_chars = char_1 + char_2

        return merged_chars


    def initialise_corpus(self, words):
        ''' Split each word into characters and count the word frequency.

            Split each word in the input word list on every character. For each
            word, store the split word in a list as the first element inside a
            tuple. Store the frequency count of the word as an integer as the
            second element of the tuple. Create a tuple for every word in this
            fashion and store the tuples in a list called 'corpus', then return
            then corpus list.

            Args:
                None.

            Returns:
                corpus (list[tuple(list, int)]):
                               A list of tuples where the first element is a
                               list of a word in the words list (where the
                               elements are the individual characters of the
                               word), and the second element is an integer
                               representing the frequency of the word in the
                               list.
        '''
        corpus = self.calculate_frequency(words)
        corpus = [(self.add_hashes(word), freq) for (word, freq) in corpus]
        return corpus

    def tokenize(self, text):
        ''' Take in some text and return a list of tokens for that text.

            Args:
                text (str): The text to be tokenized.

            Returns:
                tokens (list): The list of tokens created from the input text.
        '''
        # Create a cleaned vocabulary list without # and commas to check against
        clean_vocabulary = [word.replace('#', '').replace(',', '') for word in self.vocabulary]
        clean_vocabulary.sort(key=lambda word: len(word))
        clean_vocabulary = clean_vocabulary[::-1]

        # Break down the text into the largest tokens first, then smallest
        remaining_string = text
        tokens = []
        keep_checking = True

        while keep_checking:
            keep_checking = False
            for vocab in clean_vocabulary:
                if remaining_string.startswith(vocab):
                    tokens.append(vocab)
                    remaining_string = remaining_string[len(vocab):]
                    keep_checking = True

        if len(remaining_string) > 0:
            tokens.append(remaining_string)

        return tokens


### 3.8 - Training WordPiece  with a Toy Dataset

The WordPiece algorithm is trained below with the same toy dataset that was given to the BPE algorithm, and this time the tokens that it learns are very different. It is clear to see that WordPiece favours character combinations where the characters appear more commonly with each other than without, and so `m` and `p` are merged immediately since they only exist in the dataset together and not alone. The idea here is to force the model to consider what is being lost be merging characters together. That is, are these characters always together? In that case the characters should be merged as they are clearly one unit. Alternatively, are the characters very frequent in the corpus any way? In that case the characters may just be common and so appear next to other tokens by virtue of the abundance in the dataset [13].

&nbsp;


In [18]:
wp = WordPiece()
wp.train(words, 30)

print(f'INITIAL CORPUS:\n{wp.corpus_history[0]}\n')
for rule, corpus in list(zip(wp.merge_rules, wp.corpus_history[1:])):
    print(f'NEW MERGE RULE: Combine "{rule[0]}" and "{rule[1]}"')
    print(corpus, end='\n\n')

INITIAL CORPUS:
[(['c', '##a', '##t'], 5), (['c', '##a', '##t', '##s'], 2), (['e', '##a', '##t'], 10), (['e', '##a', '##t', '##i', '##n', '##g'], 3), (['r', '##u', '##n', '##n', '##i', '##n', '##g'], 2), (['j', '##u', '##m', '##p', '##i', '##n', '##g'], 1), (['f', '##o', '##o', '##d'], 6)]

NEW MERGE RULE: Combine "##m" and "##p"
[(['c', '##a', '##t'], 5), (['c', '##a', '##t', '##s'], 2), (['e', '##a', '##t'], 10), (['e', '##a', '##t', '##i', '##n', '##g'], 3), (['r', '##u', '##n', '##n', '##i', '##n', '##g'], 2), (['j', '##u', '##mp', '##i', '##n', '##g'], 1), (['f', '##o', '##o', '##d'], 6)]

NEW MERGE RULE: Combine "r" and "##u"
[(['c', '##a', '##t'], 5), (['c', '##a', '##t', '##s'], 2), (['e', '##a', '##t'], 10), (['e', '##a', '##t', '##i', '##n', '##g'], 3), (['ru', '##n', '##n', '##i', '##n', '##g'], 2), (['j', '##u', '##mp', '##i', '##n', '##g'], 1), (['f', '##o', '##o', '##d'], 6)]

NEW MERGE RULE: Combine "j" and "##u"
[(['c', '##a', '##t'], 5), (['c', '##a', '##t', '##s'], 2)

### 3.9 - Tokenizing using WordPiece

Now that the WordPiece algorithm has been trained (i.e. now that all have the merging rules have been found), the model can be used to tokenize any text by first splitting each of the words on every character, and then finding the largest token it can at the start of the string, then finding the largest token it can for the remaining part of the string. This process repeats until no more of the strings matches a known token from the training data and so the remaining part of the string is take as the final token.

Given the very limited training data, the tokens that the model has been able to learn are quite strange, but interesting nonetheless. With the model trained on this training data, we can see the following performance on the nonsense string `runntjumpy`. First the string is broken into `['runn','tjumpy']`, since `runn` is the largest token from the training set that can be found at the start of the word. Next, the largest token that can be found at the start of `tjumpy` given the training data is just the letter `t`, and so now the tokens are `['runn', 't', 'jumpy']`. After this, `jump` can be found and so the tokens become `['runn', 't', 'jump', 'y']`. Finally, the letter `y` does not appear in the training data, so is added as the last token.

&nbsp;

In [19]:
print(wp.tokenize('runntjumpy'))
print(wp.tokenize('runningandjumping'))
print(wp.tokenize('jumper'))
print(wp.tokenize('unknown'))

['runn', 't', 'j', 'u', 'm', 'p', 'y']
['running', 'a', 'n', 'd', 'jumping']
['jump', 'e', 'r']
['u', 'n', 'known']


### 3.10 - Unigram Tokenization


Unigram tokenizers take a different approach to BPE and WordPiece, by starting with a large vocabulary, and iteratively decreasing it until the desired size is reached, instead of the other way around.

The Unigram model uses a statistical approach, where the probability of each word or character in a sentence is considered. For example in the sentence `This is an example sentence`, it can be split into either `['This', 'is', 'an', 'example', 'sentence']` or `['T', 'h' , 'i', 's', '_i', 's', '_a', 'n','_e', 'x', 'a', 'm', 'p', 'l', 'e', '_s', 'e', 'n', 't', 'e', 'n', 'c', 'e']`. Notice how in the case where the sentence is split by characters, an underscore is prepended to the beginning of each character where a space would have been used to mark the start of a new word in the original sentence.

Each element in these lists can be considered a token, $t$, and the probability of the series of tokens, $t_1, t_2, ..., t_n$, occuring being given by:

&nbsp;

$
    P(t_1, t_2, ..., t_n) = P(t_1) \space \cdot P(t_2) \space \cdot \space ... \space \cdot \space P(t_n)
$

&nbsp;

The Unigram model is trained using the following steps:

&nbsp;

**Step 1) Construct the Corpus**

As always, the input text is given to the normalization and pre-tokenization models to create clean words. The words are then given to the Unigram model, which determines the frequency of each word, and stores this number alongside the word in a list called the corpus.

&nbsp;

**Step 2) Construct the Vocabulary**

The vocabulary size of the Unigram model starts out very large, and is iteratively decreased until the desired size is reached. To construct the initial vocabulary, find every possible substring in the corpus. For example, if the first word in the corpus is 'cats', the substrings `['c', 'a', 't', 's', 'ca', 'at', 'ts', 'cat', 'ats']` will be added to the vocabulary.

**Step 3) Calculate the Probability of Each Token**

The probability of a token is approximated by finding the number of occurrences of the token in the corpus, and dividing by the total number of token occurrences.

&nbsp;

$
    P(t) = \frac{\text{frequency of t}}{\text{frequency of all tokens}}
$

&nbsp;

**Step 4) Find All the Possible Segmentations of the Word**

Consider the case where a word from the training corpus is `cat`. This can be segemented in the following ways:

`['c', 'a', 't']`

`['ca', 't']`

`['c', 'at']`

`['cat']`

&nbsp;

**Step 5) Calculate the Approximate Probability of Each Segmentation Occuring in the Corpus**

Using the rule $P(t_1, t_2, ..., t_n) = \frac{\text{frequency of t}}{\text{frequency of all tokens}}$, the probability for each series of tokens can be calculated. As an example, this might look like:

&nbsp;

$P(c, a, t) = P(c) \space \cdot P(a) \space \cdot P(t) = \frac{30}{100} \cdot \frac{20}{100} \cdot \frac{25}{100} = 0.015$

$P(ca, t) = P(ca) \space \cdot P(t) = \frac{5}{100} \cdot \frac{20}{100} = 0.01$

$P(c, at) = P(c) \space \cdot P(at) = \frac{30}{100} \cdot \frac{15}{100} = 0.045$

$P(cat) = \frac{5}{100} = 0.05 $

&nbsp;

Since the segmentation `['ca', 't']` has the highest probability score, this is the segmentation that is used to tokenize the word. So the word `cat` will be tokenized as `['ca', 't']`. You can imagine that for longer words such as `tokenization`, splits may occur in multiple places throughout the word, for example `['token', 'iza', tion]` or `['token', 'ization]`.

&nbsp;

**Step 6) Calculate the Loss**

The term **loss** refers to a score for the model, such that if an important token is removed from the vocabulary the loss increases a lot, but if a less important token is removed the loss increases a lot. By calculating what the loss would be in the model for each token if it is was removed, it is possible to find the token that is the least useful in the vocabulary. This can be repeated iteratively until the vocabulary size has decreased to only the most useful tokens remain from the training set (corpus). The loss is given by:

&nbsp;

$
\sum{(\text{frequency of word}) \cdot (-log(P(\text{word})))}
$

&nbsp;

Once the enough characters have been removed to bring the vocabulary down to the desired size, the training is finished and the model can be used to tokenize words.


### 3.12 - Comparing BPE, WordPiece, and Unigram

Depending on the training set and the data to be tokenized, some tokenizers may perform better than others. When choosing a tokenizer for a language model, it may be best to experiment with the training set used for a specific use-case and see which gives the best results. However, there are some general tendancies of these three tokenizers that are useful to discuss.

Of the three, BPE seems to be the most popular choice for current language model tokenizers. Although in a such a rapidly changing space, this is likely to change in the future. In fact, other subword tokenizers such as SetencePiece are gaining much more popularity in recent times [14].

WordPiece seems to produce more single-word tokens compared to BPE and Unigram, but irrespective of model choice all tokenizers seem to produce fewer tokens as the vocabulary sizes increases [15].

Ultimately however, the choice of tokenizer depends on the datasets that are intended to be used with the model. Although a safe bet may be to try BPE or SentencePiece, and experiment from there.

&nbsp;

### 3.13 - Post-Processing

The final step of the tokenization pipeline is **post-processing**, and is where any final modifications can be made to the output if necessary. BERT famously uses this step to add two additional types of tokens:

* `[CLS]` - this token stands for 'classification', and is used to mark beginning of the input text. This is required since one of the training tasks for BERT is sentence classification (hence the name of the token).

* `[SEP]` - this token stands for 'separation'  and is used to separate sentences in the input. This is useful for many tasks that BERT performs, including handling multiple instructions at once in the same prompt [16].

&nbsp;

<h2 align="center">Section 4 - Tokenizers in Python Libraries</h2>

### 4.1 - Overview of the Hugging Face `tokenizers` Library

Hugging Face provides the`tokenizers` library which is available in several programming languages, including bindings for Python. The library contains a generic `Tokenizer` class which allows the user to work with pre-trained models, the full list of which can be found on the Hugging Face website [17]. In addition, the library also contains four pre-made but untrained models that the user can train with their own data [18]. These are useful for creating specific tokenizers that are tuned to a particular type of document. The cells below show exmaples of working with both the pre-trained and untrained tokenizers in Python.

&nbsp;

### 4.2 - Using Pre-Trained Tokenizers

The `tokenizers` library makes it very easy to use a pre-trained tokenizer. Simply import the `Tokenizer` class, and called the `from_pretrained` method and pass in the name of the model to use the tokenizer from. A list of models is given in [17].

&nbsp;

In [31]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained('bert-base-cased')

### 4.3 - Training Pre-made Tokenizers

To use one of the pre-made but untrained tokenizers, simply import the desired model from the `tokenizers` library, and create an instance of the model class. As stated above, there for four models included in the library:

&nbsp;

* `BertWordPieceTokenizer` - The famous Bert tokenizer, using WordPiece

* `CharBPETokenizer` - The original BPE

* `ByteLevelBPETokenizer` - The byte level version of the BPE

* `SentencePieceBPETokenizer` - A BPE implementation compatible with the one used by SentencePiece

&nbsp;

To train the model, use the `train` method and pass the path to a file (or list of paths to files) containing the training data. Once trained, the model can then be used to tokenize some text using the `encode` method. Finally, the trained tokenizer can be saved using the `save` method so that training does not have to be performed again. The cell below shows some example code which has been adapted from the examples available on the Hugging Face Tokenizers GitHub page [17].

&nbsp;

In [20]:
# Import a tokenizer
from tokenizers import BertWordPieceTokenizer, CharBPETokenizer, ByteLevelBPETokenizer, SentencePieceBPETokenizer

# Instantiate the model
tokenizer = CharBPETokenizer()

# Train the model
tokenizer.train(['./path/to/files/1.txt', './path/to/files/2.txt'])

# Tokenize some text
encoded = tokenizer.encode('I can feel the magic, can you?')

# Save the model
tokenizer.save('./path/to/directory/my-bpe.tokenizer.json')

### 4.4 - Building Custom Tokenizers

The `tokenizer` library also provides the components to construct a custom tokenizer very quickly, without having to implement an entire model from scratch as shown in the previous section. The cell below shows an example taken from the Hugging Face GitHub page [18] which shows how to customise the pre-tokenization and decoding stage of the tokenizer. In this case, prefix spaces were added in the pre-tokenization stage, and the decoder chosen was the ByteLevel decoder. A full list of customisation options is available in the Hugging Face documentation [19].

&nbsp;


In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

# Initialize a tokenizer
tokenizer = Tokenizer(models.BPE())

# Customize pre-tokenization and decoding
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

# And then train
trainer = trainers.BpeTrainer(
    vocab_size=20000,
    min_frequency=2,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)
tokenizer.train([
    "./path/to/dataset/1.txt",
    "./path/to/dataset/2.txt",
    "./path/to/dataset/3.txt"
], trainer=trainer)

# And Save it
tokenizer.save("byte-level-bpe.tokenizer.json", pretty=True)

<h2 align="center">Section 5 - Conclusion</h2>

The tokenization pipeline is a crucial part of any language model, and careful consideration should be given when deciding which type of tokenizer to use. Nowadays, many of these decisions have been made for us by the developers of libraries such as those provided by Hugging Face. These allow users to quickly train and used language models with custom data. However, a solid understanding of tokenization methods is invaluable for fine-tuning models, and squeezing out additional performance on different datasets.

&nbsp;


<h2 align="center">Section 6 - Glossary</h2>

**Byte-Level BPE**
> A trick used by the GPT-2 and RoBERTa models (among others) whereby characters in the training data for a BPE algorithm are analysis using their bytes instead of the Unicode representation. This eliminates out of vocabulary words since every word can be tokenized.

**Byte Pair Encoding (BPE)**
> A subword tokenization algorithm that is a commonly-used in many transformer models such as Open AI's GPT and GPT-2 models, BART, and many others. The BPE algorithm aims to decompose a string of text into subword units that appear frequently in a reference corpus (the text used to train the tokenization model)

**Character-based tokenization**
> A tokenization method in which the a sentence/sentences are split on each character, including letters, numbers, and special characters such as punctuation.

**Encoding**
> The process of replacing the textual tokens with a number representation.

**Exploding Vocabulary Problem**
> An issue with word-based tokenization methods in which the number of words in a models vocabulary increases to very high levels (e.g. greater than the number of words in the English language).

**Decoding**
> The process of converting encoded tokens back into text.

**Normalization**
> The process of cleaning text before it is split into tokens. This includes converting each character to lowercase, removing accents from characters (e.g. 'é' becomes 'e'), removing unnecessary whitespace, and so on.

**Out-of-Vocabulary (OOV)**
> Words which are not found in the vocabulary of a model.

**Post-Processing**
> The final step of the tokenization pipeline, and is where any final modifications can be made to the output if necessary.

**Pre-tokenization**
> The first splitting of the raw text in the tokenization pipeline. The split is performed to give an upper bound to what the final tokens could be at the end of the pipeline. That is, a sentence can be split into words in the pre-tokenization step, then in the model step some of these words may be split further according to the tokenization method (e.g. subword-based).

**Subword-based tokenization**
> A tokenization methods which aims to achieve the benefits of both word-based and character-based methods, by splitting sentences within words.

**Token**
> An instance of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing.

**Tokenization pipeline**
> The series of actions that are taken to convert raw text into tokens. The steps of this pipeline are: normalization, pre-tokenization, model post-processing.

**Tokenizer**
> A model which converts strings of text into tokens. The tokens can then be associated with a number, which allows human lanuage to be expressed in a form computers can understand.

**Unigram**
> A subword tokenization method that starts with a large vocabulary, which iteratively decreases in size until the desired size is reached, unlike BPE and WordPiece which iteratively increase their vocavulary size.

**Vocabulary**
> The complete set of possible tokens in a tokenization model.

**Word-Based Tokenization**
> A tokenization method in which a sentence/sentences is split into words by splitting on each space in the sentence (sometimes called 'whitespace-based tokenization), or by a similar set of rules (such as punctuation-based tokenization, treebank tokenization, etc).

**WordPiece**
> A subword tokenization method developed by Google for their seminal BERT model, and was used in derivative models such as DistilBERT and MobileBERT.

&nbsp;






<h2 align="center">Section 7 - Further Reading</h2>

&nbsp;

[1] - Token Definition [Stanford NLP Group](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html#:~:text=A%20token%20is%20an%20instance,useful%20semantic%20unit%20for%20processing.)

[2] Word Tokenizers - [Towards Data Science](https://towardsdatascience.com/top-5-word-tokenizers-that-every-nlp-data-scientist-should-know-45cc31f8e8b9#:~:text=Tokenization%20is%20the%20process%20of,I%E2%80%9D%20and%20%E2%80%9Cwon%E2%80%9D.)

[3] Tokenizers - [Hugging Face](https://huggingface.co/docs/transformers/tokenizer_summary)

[4] TransformerXL Paper - [ArXiv](https://arxiv.org/abs/1901.02860)

[5] Word-Based, Subword, and Character-Based Tokenizers - [Towards Data Science](https://towardsdatascience.com/word-subword-and-character-based-tokenization-know-the-difference-ea0976b64e17)

[6] A Comprehensive Guide to Subword Tokenizers - [Towards Data Science](https://towardsdatascience.com/a-comprehensive-guide-to-subword-tokenisers-4bbd3bad9a7c)

[7] The Tokenization Pipeline - [Hugging Face](https://huggingface.co/docs/tokenizers/pipeline)

[8] Pre-tokenizers - [Hugging Face](https://huggingface.co/docs/tokenizers/api/pre-tokenizers)

[9] Language Models are Unsupervised Multitask Learners - [OpenAI](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)

[10] BART Model for Text Autocompletion in NLP - [Geeks for Geeks](https://www.geeksforgeeks.org/bart-model-for-text-auto-completion-in-nlp/)

[11] Byte Pair Encoding - [Hugging Face](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt#byte-pair-encoding-tokenization)

[12] WordPiece Tokenization - [Hugging Face](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)

[13] Two minutes NLP — A Taxonomy of Tokenization Methods - [Medium](https://medium.com/nlplanet/two-minutes-nlp-a-taxonomy-of-tokenization-methods-60e330aacad3#:~:text=In%20contrast%20to%20BPE%2C%20WordPiece,to%20ensure%20it's%20worth%20it.)

[14] Subword Tokenizer Comparison - [Vinija AI](https://vinija.ai/nlp/tokenizer/)

[15] [A Comprehensive Analysis of Subword Tokenizers for
Morphologically Rich Languages - [Marmara University](https://www.cmpe.boun.edu.tr/~gungort/theses/A%20Comprehensive%20Analysis%20of%20Subword%20Tokenizers%20for%20Morphologically%20Rich%20Languages.pdf)

[16] How BERT Leverage Attention Mechanism and Transformer to Learn Word Contextual Relations - [Medium](https://towardsdatascience.com/how-bert-leverage-attention-mechanism-and-transformer-to-learn-word-contextual-relations-5bbee1b6dbdb#:~:text=BERT%20use%20three%20embeddings%20to,separate%20segment%20(or%20sentence).)

[17] List of Pretrained Models - [Hugging Face](https://huggingface.co/transformers/v3.3.1/pretrained_models.html)

[18] Hugging Face Tokenizers Library - [GitHub](https://github.com/huggingface/tokenizers/blob/main/bindings/python/README.md)

[19] Pre-Tokenization Documentation - [Hugging Face](https://huggingface.co/docs/tokenizers/api/pre-tokenizers)